## Imports and setup

In [ ]:
import time
import os
import csv
from random import choices
from itertools import product
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

color_sets = [
    ['R', 'G', 'B', 'Y'],
    ['R', 'G', 'B', 'Y', 'O'],
    ['R', 'G', 'B', 'Y', 'O', 'P'],
]

slot_configs = [3, 4, 5]
run_counts = [10, 50, 100]

## Feedback + Strategies

In [ ]:
def get_feedback(secret_code, guess):
    black = sum(s == g for s, g in zip(secret_code, guess))
    white = sum(min(secret_code.count(c), guess.count(c)) for c in set(guess)) - black
    return black, white

def filter_candidates(candidates, guess, feedback):
    black, white = feedback
    return [c for c in candidates if get_feedback(c, guess) == (black, white)]

def calculate_color_probabilities(candidates, COLORS, CODE_LENGTH):
    color_counts = {color: 0 for color in COLORS}
    for candidate in candidates:
        for color in candidate:
            color_counts[color] += 1
    total = len(candidates) * CODE_LENGTH
    return {color: color_counts[color] / total for color in COLORS}

## Stretegies

In [ ]:
def attempts_by_random(secret_code, all_candidates, CODE_LENGTH, MAX_ATTEMPTS):
    candidates = all_candidates.copy()
    attempts = 0
    while attempts < MAX_ATTEMPTS and candidates:
        guess = choices(candidates, k=1)[0]
        feedback = get_feedback(secret_code, guess)
        if feedback[0] == CODE_LENGTH:
            return attempts + 1
        candidates = filter_candidates(candidates, guess, feedback)
        attempts += 1
    return MAX_ATTEMPTS + 1

def attempts_by_fixed_then_random(secret_code, all_candidates, CODE_LENGTH, MAX_ATTEMPTS):
    candidates = all_candidates.copy()
    attempts = 0
    initial_guess = 'R' * CODE_LENGTH  # For example, all 'R's, or you can customize
    feedback = get_feedback(secret_code, initial_guess)
    attempts += 1
    if feedback[0] == CODE_LENGTH:
        return attempts
    candidates = filter_candidates(candidates, initial_guess, feedback)
    
    while attempts < MAX_ATTEMPTS and candidates:
        guess = choices(candidates, k=1)[0]
        feedback = get_feedback(secret_code, guess)
        if feedback[0] == CODE_LENGTH:
            return attempts + 1
        candidates = filter_candidates(candidates, guess, feedback)
        attempts += 1
        
    return MAX_ATTEMPTS + 1

def attempts_by_greedy_approach(secret_code, all_candidates, COLORS, CODE_LENGTH, MAX_ATTEMPTS):
    candidates = all_candidates.copy()
    attempts = 0
    while attempts < MAX_ATTEMPTS and candidates:
        color_probs = calculate_color_probabilities(candidates, COLORS, CODE_LENGTH)
        guess = max(candidates, key=lambda c: sum(color_probs[color] for color in c))
        feedback = get_feedback(secret_code, guess)
        if feedback[0] == CODE_LENGTH:
            return attempts + 1
        candidates = filter_candidates(candidates, guess, feedback)
        attempts += 1
    return MAX_ATTEMPTS + 1

def score_guess(guess, candidates):
    feedback_counts = defaultdict(int)
    for code in candidates:
        feedback = get_feedback(code, guess)
        feedback_counts[feedback] += 1
    return max(feedback_counts.values())

def select_minimax_guesses(all_candidates, candidates):
    scores = {}
    for guess in all_candidates:
        scores[guess] = score_guess(guess, candidates)
    min_score = min(scores.values())
    return [guess for guess, score in scores.items() if score == min_score]

def get_next_guess(minimax_guesses, candidates, all_candidates):
    for guess in minimax_guesses:
        if guess in candidates:
            return guess
    for guess in minimax_guesses:
        if guess in all_candidates:
            return guess
    return minimax_guesses[0]  # fallback

def attempts_by_knuth_algorithm(secret_code, all_candidates, CODE_LENGTH, MAX_ATTEMPTS):
    candidates = all_candidates.copy()
    guess = 'RRRGGG'[:CODE_LENGTH]  # Adjust initial guess to CODE_LENGTH
    attempts = 1

    while attempts <= MAX_ATTEMPTS:
        feedback = get_feedback(secret_code, guess)
        if feedback[0] == CODE_LENGTH:
            return attempts
        candidates = filter_candidates(candidates, guess, feedback)
        minimax_guesses = select_minimax_guesses(all_candidates, candidates)
        guess = get_next_guess(minimax_guesses, candidates, all_candidates)
        attempts += 1

    return MAX_ATTEMPTS + 1

## Benchmarking + Logging

In [27]:
RESULTS_FILE = 'results.csv'

def log_to_csv(row):
    file_exists = os.path.exists(RESULTS_FILE)
    with open(RESULTS_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['Algorithm', 'Slots', 'NumColors', 'Runs', 'AvgAttempts', 'AvgTime'])
        writer.writerow(row)

def benchmark_algorithms(RUNS, SLOTS, COLORS):
    CODE_LENGTH = SLOTS
    MAX_ATTEMPTS = 10
    all_candidates = [''.join(p) for p in product(COLORS, repeat=SLOTS)]

    stats = { 'Random': [], 'Greedy': [], 'fixed_start': [], 'Knuth': [] }
    times = { 'Random': [], 'Greedy': [], 'fixed_start': [], 'Knuth': [] }

    for _ in range(RUNS):
        secret_code = ''.join(choices(COLORS, k=CODE_LENGTH))
        for algo, func in [
            ('Random', attempts_by_random),
            ('Greedy', attempts_by_greedy_approach),
            ('fixed_start', attempts_by_fixed_then_random),
            ('Knuth', attempts_by_knuth_algorithm)
        ]:
            start = time.time()
            if algo in ['Greedy']:
                attempts = func(secret_code, all_candidates, COLORS, CODE_LENGTH, MAX_ATTEMPTS)
            else:
                attempts = func(secret_code, all_candidates, CODE_LENGTH, MAX_ATTEMPTS)
            end = time.time()
            stats[algo].append(attempts)
            times[algo].append(end - start)

    print(f"\n📊 Configuration: SLOTS={SLOTS}, COLORS={len(COLORS)}, RUNS={RUNS}")
    print("=" * 60)
    print(f"{'Algorithm':<12}{'Avg Attempts':<15}{'Avg Time (s)':<15}")
    print("-" * 60)
    for algo in stats:
        avg_attempts = round(mean(stats[algo]), 2)
        avg_time = round(mean(times[algo]), 4)
        print(f"{algo:<12}{avg_attempts:<15}{avg_time:<15}")
        log_to_csv([algo, SLOTS, len(COLORS), RUNS, avg_attempts, avg_time])
    print("=" * 60)


## Plotting

In [28]:
def plot_results(csv_file=RESULTS_FILE):
    import pandas as pd
    sns.set(style="whitegrid")
    df = pd.read_csv(csv_file)

    # Attempts vs Algorithm
    plt.figure(figsize=(8, 5))
    sns.barplot(data=df, x='Algorithm', y='AvgAttempts', ci=None)
    plt.title("Average Attempts by Algorithm")
    plt.tight_layout()
    plt.savefig("avg_attempts_per_algorithm.png")
    plt.close()

    # Time vs Algorithm
    plt.figure(figsize=(8, 5))
    sns.barplot(data=df, x='Algorithm', y='AvgTime', ci=None)
    plt.title("Average Time by Algorithm")
    plt.tight_layout()
    plt.savefig("avg_time_per_algorithm.png")
    plt.close()

    # Attempts vs Slots
    plt.figure(figsize=(8, 5))
    sns.lineplot(data=df, x='Slots', y='AvgAttempts', hue='Algorithm', marker='o')
    plt.title("Avg Attempts vs Slot Count")
    plt.tight_layout()
    plt.savefig("attempts_vs_slots.png")
    plt.close()

    # Attempts vs NumColors
    plt.figure(figsize=(8, 5))
    sns.lineplot(data=df, x='NumColors', y='AvgAttempts', hue='Algorithm', marker='o')
    plt.title("Avg Attempts vs Number of Colors")
    plt.tight_layout()
    plt.savefig("attempts_vs_colors.png")
    plt.close()

    print("✅ Plots saved successfully.")

## Run Benchmarks

In [ ]:
for COLORS in color_sets:
    for SLOTS in slot_configs:
        for RUNS in run_counts:
            benchmark_algorithms(RUNS, SLOTS, COLORS)

# Plot at the end
plot_results()

In [29]:
plot_results()

C:\Users\tupti\AppData\Local\Temp\ipykernel_18520\994992885.py:8: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(data=df, x='Algorithm', y='AvgAttempts', ci=None)
C:\Users\tupti\AppData\Local\Temp\ipykernel_18520\994992885.py:16: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(data=df, x='Algorithm', y='AvgTime', ci=None)
c:\Users\tupti\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\tupti\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\tupti\anaconda3\

✅ Plots saved successfully.
